Alunos:

*   Andréa Fonseca
*   Fábio Cardoso
*   Eduardo Leite



In [1]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


## Retrieval Augmented Generation (RAG)

### Carregando Documentos - Loading

In [2]:
!pip install langchain_community langchain_openai faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 722.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.9/393.9 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.1/149.1 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.

In [3]:
# https://python.langchain.com/v0.2/docs/how_to/#document-loaders
# https://python.langchain.com/v0.2/docs/integrations/document_loaders/

import bs4
from langchain_community.document_loaders import WebBaseLoader

# Filtra o conteúdo da página por uma classe específica
bs4_strainer = bs4.SoupStrainer(class_=("mw-body-content"))

# Carrega o conteúdo da página
loader = WebBaseLoader(
    web_paths=("https://pt.wikipedia.org/wiki/Centro_de_Estudos_e_Sistemas_Avan%C3%A7ados_do_Recife",),
    bs_kwargs={"parse_only": bs4_strainer},
)

# Carrega o conteúdo da página
docs = loader.load()

len(docs[0].page_content)

6915

In [50]:
docs

[Document(metadata={'source': 'https://atosoficiais.com.br/anp/resolucao-n-791-2019-dispoe-sobre-a-individualizacao-das-metas-compulsorias-anuais-de-reducao-de-emissoes-de-gases-causadores-do-efeito-estufa-para-a-comercializacao-de-combustiveis-no-ambito-da-politica-nacional-de-biocombustiveis-renovabio'}, page_content='\n\n\n\n\n\n\nAtualizações desta norma\n\n\n\n\n\t\t\t\t\t\tResolução Nº 974/2024\n\t\t\t\t\n\n\n\n\n\n\t\t\t\t\t\tResolução Nº 921/2023\n\t\t\t\t\n\n\n\n\n\n\t\t\t\t\t\tPauta Reunião de Diretoria Nº 1113/2023\n\t\t\t\t\n\n\n\n\n\n\t\t\t\t\t\tResolução de Diretoria Nº 147/2023\n\t\t\t\t\n\n\n\n\n\n\t\t\t\t\t\tPauta Reunião de Diretoria Nº 1112/2023\n\t\t\t\t\n\n\n\n\n\n\t\t\t\t\t\tResolução Nº 843/2021\n\t\t\t\t\n\n\n\n\n\n\t\t\t\t\t\tResolução de Diretoria Nº 280/2021\n\t\t\t\t\n\n\n\n\n  AGÊNCIA NACIONAL DO PETRÓLEO, GÁS NATURAL E BIOCOMBUSTÍVEIS   RESOLUÇÃO N 791, DE 12 DE JUNHO DE 2019(Vide Resolução de Diretoria nº 147/2023)(Vide Pauta Reunião de Diretoria nº 1112/

In [4]:
print(docs[0].page_content[500:1000])

cife, também conhecido por seu acrônimo CESAR, é um centro de pesquisa e inovação sem fins lucrativos com sede na cidade do Recife, Pernambuco e filiais em Sorocaba, Curitiba e Manaus.[1] O CESAR foi fundado em 1996 por três professores do Centro de Informática da UFPE, Silvio Meira, Fábio Silva e Ismar Kaufman, como forma de aproximar a academia do mercado.[2]
Em 2019, o centro conta com mais de 600 funcionários e em 2018 seu faturamento foi da ordem de R$ 100 milhões.[1][3][4][5]
O CESAR é par


### Dividindo Documentos - Splitting/Chunking

In [5]:
# https://python.langchain.com/v0.2/docs/how_to/#text-splitters

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

10

In [51]:
all_splits

[Document(metadata={'source': 'https://pt.wikipedia.org/wiki/Centro_de_Estudos_e_Sistemas_Avan%C3%A7ados_do_Recife', 'start_index': 1}, page_content='Centro de Estudos e Sistemas Avançados do Recife(CESAR)\n\n\nCentro de Estudos e Sistemas Avançados do Recife\n\n\nCentro de Estudos e Sistemas Avançados do Recife\n\n\nTipo\n\nInstituto de pesquisa sem fins lucrativos\n\n\nFundação\n\n20 de maio de 1996 (28\xa0anos)\n\n\nSede\n\nRecife, PE\n\n\nFundadores\n\nSilvio Meira, Fábio Silva e Ismar Kaufman\n\n\nOrganização\n\nEduardo Peixoto (CEO)\nKarla Godoy (COO)\n\n\n\nEmpregados\n\n600+ (2019) [1]\n\n\nSítio oficial\n\nhttp://www.cesar.org.br'),
 Document(metadata={'source': 'https://pt.wikipedia.org/wiki/Centro_de_Estudos_e_Sistemas_Avan%C3%A7ados_do_Recife', 'start_index': 258}, page_content='Recife, PE\n\n\nFundadores\n\nSilvio Meira, Fábio Silva e Ismar Kaufman\n\n\nOrganização\n\nEduardo Peixoto (CEO)\nKarla Godoy (COO)\n\n\n\nEmpregados\n\n600+ (2019) [1]\n\n\nSítio oficial\n\nhttp:/

In [6]:
print(all_splits[2].page_content)

Área Educacional[editar | editar código-fonte]
Além de atuar como centro de pesquisa e inovação, o CESAR criou um braço educacional, a CESAR School, oferecendo cursos de graduação, mestrados e doutorados profissionais.[4]
O centro iniciou sua atuação na área educacional em 2007 quando iniciou a oferta do mestrado profissional em Engenharia de Software, que foi avaliado pela CAPES em 2017 como um dos dois melhores mestrados profissionais na área de computação do país.[8] Em 2013 foi autorizada a abertura do segundo mestrado profissional, dessa vez com ênfase em Design de Artefatos Digitais.[9] A partir de 2016 o mestrado em Design passou a também ser oferecido na unidade de Manaus.[10]


### Indexando - Store

In [7]:
# https://python.langchain.com/v0.2/docs/how_to/embed_text/

from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

vectorstore = FAISS.from_documents(all_splits, OpenAIEmbeddings())

In [8]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("Onde está localizado o CESAR?")

len(retrieved_docs)

6

In [11]:
print(retrieved_docs[3].page_content)

Em 2016 o Ministério da Educação autorizou o CESAR a ofertar cursos de graduação[11] e começaram a ser ofertados os cursos de Ciência da Computação e Design, com  abertura da primeira turma no primeiro semestre de 2018.[12] A unidade de ensino superior usa a metodologia de aprendizagem baseada em problemas, que permite o contato com demandas reais da sociedade desde os primeiros dias do curso.[13]
Em 2019 o CESAR se tornou a primeira instituição do país a oferecer um curso de doutorado na modalidade profissional na área de engenharia de software.[4][14]


### Buscando e Recuperando Informações - Retrieve

In [12]:
from langchain_core.prompts import ChatPromptTemplate

system_template = """Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.

Pergunta: {question}

Contexto: {context}

Resposta:
"""

prompt_template = ChatPromptTemplate.from_template(system_template)

In [13]:
example_messages = prompt_template.invoke({
    "context": "algum contexto",
    "question": "alguma pergunta"
})

print(example_messages.to_messages())

[HumanMessage(content='Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.\n\nPergunta: alguma pergunta\n\nContexto: algum contexto\n\nResposta:\n')]


### Gerando Respostas - Generate

In [14]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [15]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

In [16]:
for chunk in rag_chain.stream("Onde fica o CESAR?"):
    print(chunk, end="", flush=True)

O CESAR fica em Recife, Pernambuco. Ele é um centro de pesquisa e inovação com a sede na cidade.

## Exercícios

### Exercício 1
Faça um RAG com um pequeno arquivo de texto, contendo informações que, certamente, a LLM não conheça. Você deverá construir o arquivo e enviar para o ambiente de execução. Escolha a forma de chunking apropriada para o seu documento.

In [32]:
import requests
from bs4 import BeautifulSoup

In [84]:
url = 'https://atosoficiais.com.br/anp/resolucao-n-791-2019-dispoe-sobre-a-individualizacao-das-metas-compulsorias-anuais-de-reducao-de-emissoes-de-gases-causadores-do-efeito-estufa-para-a-comercializacao-de-combustiveis-no-ambito-da-politica-nacional-de-biocombustiveis-renovabio'
response = requests.get(url)
html_content = response.text

In [85]:
html_content

'<!DOCTYPE html>\n<html lang="pt-br">\n  <head>\n\t<meta charset="utf-8">\n  \t<meta http-equiv="X-UA-Compatible" content="IE=edge">\n  \t<meta name="viewport" content="width=device-width, initial-scale=1">\n\t<meta name="theme-color" content="#0c8fcc" />\n\t<link rel=\'shortcut icon\' type=\'image/x-icon\' href=\'/favicon.ico\' />\n\n  \t<title>Resolução 791 2019 da ANP - Agência Nacional do Petróleo, Gás Natural e Biocombustíveis BR</title>\n\n  \t<link href="/css/bootstrap.min.css?v=1" rel="stylesheet">\n  \t<link href="/css/screen.css?v=2" media="screen, projection" rel="stylesheet" type="text/css" />\n\t<link href="/css/institucional.css?v=1" media="screen, projection" rel="stylesheet" type="text/css" />\n\t<link rel="stylesheet" href="https://use.fontawesome.com/releases/v5.5.0/css/all.css?v=1" integrity="sha384-B4dIYHKNBt8Bc12p+WXckhzcICo0wtJAoU8YZTY5qE0Id1GSseTk6S+L3BlXeVIU" crossorigin="anonymous">\n\t<link href="/css/client.css?v=1" rel="stylesheet" type="text/css" />\n\t\n\n

In [47]:
soup = BeautifulSoup(html_content, 'html.parser')

In [60]:
# verifica se o status da norma é "Norma em vigor"
if soup.find('div', class_='btn btn-default btn-lg content-block-header-box status-norma'):
  for s_tag in soup.find_all('s'):
    s_tag.decompose()

  article = soup.find('article', id="lei")
else:
  print("A norma não está em vigor")

In [86]:
article

<article id="lei">
<div class="printable">
<img src="/img/brasao-uniao.png" style="width: 80px; display: none">
</img></div>
<!-- lei -->
<div class="hidden-print">
<p>
<strong>Atualizações desta norma</strong>
</p>
<p>
<a href="/anp/resolucao-n-974-2024-altera-a-resolucao-anp-no-791-de-12-de-junho-de-2019-que-dispoe-sobre-a-individualizacao-das-metas-compulsorias-anuais-de-reducao-de-emissoes-de-gases-geradores-do-efeito-estufa-para-a-comercializacao-de-combustiveis-no-ambito-da-politica-nacional-de-biocombustiveis-renovabio-para-incluir-hipotese-de-reducao-das-metas-em-decorrencia-da-publicacao-da-lei-no-14-592-de-2023?origin=instituicao">
<span rel="title">
						Resolução Nº 974/2024
				</span>
</a>
</p>
<p>
<a href="/anp/resolucao-n-921-2023-altera-a-resolucao-anp-no-791-de-12-de-junho-de-2019-que-dispoe-sobre-a-individualizacao-das-metas-compulsorias-anuais-de-reducao-de-emissoes-de-gases-causadores-do-efeito-estufa-para-a-comercializacao-de-combustiveis-no-ambito-da-politica-na

In [87]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema import Document

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents([Document(page_content=str(article))])

len(all_splits)

93

In [88]:
print(all_splits[7])

page_content='data-ambito="4" data-id="162552" data-original-title=" Data da Norma: 02.03.2023 - Aprovação da Ata e das Resoluções de Diretoria da 1.111ª Reunião de Diretoria, e das Resoluções de Diretoria referentes aos Circuitos Deliberativos nº s 23 a 25, 27, 28 e 30 a 58" data-toggle="tooltip" href="https://atosoficiais.com.br/anp/pauta-reuniao-de-diretoria-n-1112-2023-aprovacao-da-ata-e-das-resolucoes-de-diretoria-da-1111-reuniao-de-diretoria-e-das-resolucoes-de-diretoria-referentes-aos-circuitos-deliberativos-n-s-23-a-25-27-28-e-30-a-58" rel="tooltip">1112</a>/2023 e nº <a class="link_law" data-ambito="4" data-id="172538" data-original-title=" Data da Norma: 29.03.2023 - Aprovação da Ata e das Resoluções de Diretoria da 1.112ª Reunião de Diretoria, e das Resoluções de Diretoria referentes aos Circuitos Deliberativos nº s 59 a 68, 70 a 93, 96 a 108 e 116." data-toggle="tooltip"' metadata={'start_index': 5095}


In [89]:
vectorstore = FAISS.from_documents(all_splits, OpenAIEmbeddings())

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [90]:
system_template = """Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo quatro frases e mantenha a resposta concisa e fale apenas o necessário.

Pergunta: {question}

Contexto: {context}

Resposta:
"""

prompt_template = ChatPromptTemplate.from_template(system_template)

In [91]:
llm = ChatOpenAI(model="gpt-4o-mini")

In [92]:
rag_chain = (
     {"context": retriever, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

In [98]:
for chunk in rag_chain.stream("Caso a meta anual individual não seja cumprida, o que acontece?"):
    print(chunk, end="", flush=True)

Caso a meta anual individual não seja cumprida, a multa será aplicada e a quantidade de CBIOs não cumprida deverá ser acrescida à meta do ano seguinte. Além disso, se o não cumprimento ultrapassar quinze por cento da meta, uma multa será cobrada sobre todo o percentual de não cumprimento. O pagamento da multa não isenta o distribuidor do cumprimento da meta.